In [95]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [97]:
mkdir Brain_tumor

In [98]:
cd Brain_tumor

/content/drive/MyDrive/Brain_tumor


In [100]:
!unzip archive.zip

Archive:  archive.zip
  inflating: brain_tumor_dataset/no/1 no.jpeg  
  inflating: brain_tumor_dataset/no/10 no.jpg  
  inflating: brain_tumor_dataset/no/11 no.jpg  
  inflating: brain_tumor_dataset/no/12 no.jpg  
  inflating: brain_tumor_dataset/no/13 no.jpg  
  inflating: brain_tumor_dataset/no/14 no.jpg  
  inflating: brain_tumor_dataset/no/15 no.jpg  
  inflating: brain_tumor_dataset/no/17 no.jpg  
  inflating: brain_tumor_dataset/no/18 no.jpg  
  inflating: brain_tumor_dataset/no/19 no.jpg  
  inflating: brain_tumor_dataset/no/2 no.jpeg  
  inflating: brain_tumor_dataset/no/20 no.jpg  
  inflating: brain_tumor_dataset/no/21 no.jpg  
  inflating: brain_tumor_dataset/no/22 no.jpg  
  inflating: brain_tumor_dataset/no/23 no.jpg  
  inflating: brain_tumor_dataset/no/24 no.jpg  
  inflating: brain_tumor_dataset/no/25 no.jpg  
  inflating: brain_tumor_dataset/no/26 no.jpg  
  inflating: brain_tumor_dataset/no/27 no.jpg  
  inflating: brain_tumor_dataset/no/28 no.jpg  
  inflating: brain

In [101]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D, Dropout

from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.__version__)

2.18.0


In [102]:
img_width = 128
img_height = 128

In [103]:
!pip install split_folders

In [104]:
import splitfolders

splitfolders.ratio("/content/drive/MyDrive/Brain_tumor", output="output", seed=1337, ratio=(.8, .1, .1), group_prefix=None) # default valuesode\Projects\BrainTumorDetection_CNN\

Copying files: 253 files [00:10, 25.22 files/s]


In [105]:
train_datagen=ImageDataGenerator(rescale=1/255.0)
test_datagen=ImageDataGenerator(rescale=1/255.0)

In [106]:
train_data_generator=train_datagen.flow_from_directory(directory='/content/drive/MyDrive/Brain_tumor/output/train',
                                                       target_size=(img_height,img_width),
                                                       class_mode='categorical',
                                                       batch_size=16)

Found 202 images belonging to 4 classes.


In [107]:
validation_data_generator=test_datagen.flow_from_directory(directory='output/test',
                                                     target_size=(img_height,img_width),
                                                     class_mode='categorical',
                                                     batch_size=16)

Found 27 images belonging to 4 classes.


In [109]:
test_data_generator=test_datagen.flow_from_directory(directory='output/val',
                                                        target_size=(img_height,img_width),
                                                        class_mode='categorical',
                                                        batch_size=16)

Found 24 images belonging to 4 classes.


In [111]:

from tensorflow.keras import Input

model = Sequential([
    Input(shape=(img_height, img_width, 3)),  # Input layer
    Conv2D(16, (3, 3), activation='relu'),
    MaxPool2D(2, 2),
    Dropout(0.2),

    Conv2D(32, (3, 3), activation='relu'),
    MaxPool2D(2, 2),
    Dropout(0.3),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])


In [112]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [113]:
from tensorflow.keras.applications import VGG16 #Densenet Resnet efficientnet inception xception
#transformers---attention based networks
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
def prepare_model():
    # Load VGG16 with pre-trained weights from ImageNet and exclude the top layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    # Freeze the base model to retain pre-trained features during the initial training phase
    base_model.trainable = False

    # Add custom layers on top of the base VGG16 model
    model = Sequential()
    model.add(base_model)  # Add the VGG16 convolutional base

    # Add a global average pooling layer
    model.add(GlobalAveragePooling2D())

    # Add a fully connected layer
    model.add(Dense(16, activation='relu'))

    # Add the final output layer for multi-class classification (4 classes)
    model.add(Dense(4, activation='softmax'))

    # Compile the model
    model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=['accuracy'])

    return model

In [116]:
model.fit(train_data_generator,
                              steps_per_epoch = len(train_data_generator),
                              epochs = 5,
                              validation_data = validation_data_generator,
                              validation_steps = len(validation_data_generator))

Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 302ms/step - accuracy: 0.9969 - loss: 0.0449 - val_accuracy: 0.9259 - val_loss: 0.2248
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 445ms/step - accuracy: 0.9830 - loss: 0.0650 - val_accuracy: 0.9259 - val_loss: 0.2996
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 281ms/step - accuracy: 0.9935 - loss: 0.0397 - val_accuracy: 0.9259 - val_loss: 0.3709
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 459ms/step - accuracy: 0.9900 - loss: 0.0384 - val_accuracy: 0.9259 - val_loss: 0.3618
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 271ms/step - accuracy: 0.9954 - loss: 0.0360 - val_accuracy: 0.9259 - val_loss: 0.3249


In [117]:
# Evaluate the Model
score = model.evaluate(test_data_generator, steps=len(test_data_generator))
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.7569 - loss: 0.6434 
Test loss: 0.5719966888427734
Test accuracy: 0.7916666865348816


In [118]:
import numpy as np
from keras.preprocessing import image
from PIL import Image

In [119]:

# Prepare the image for prediction
import os
img_path1 = os.path.join('output', 'val', 'yes', 'Y2.jpg')  # Adjusted for relative path
img_yes= image.load_img(img_path1, target_size=(img_height, img_width))
img_arr = image.img_to_array(img_yes)
img_arr = np.expand_dims(img_arr, axis=0)
result=model.predict(img_arr)
result


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step


array([[0., 0., 0., 1.]], dtype=float32)

In [121]:
# Prepare the image for prediction
import os
img_path2 = os.path.join('output', 'val', 'no', '18 no.jpg')  # Adjusted for relative path
img_no = image.load_img(img_path2, target_size=(img_height, img_width))
img_arr = image.img_to_array(img_no)
img_arr = np.expand_dims(img_arr, axis=0)
result=model.predict(img_arr)
result


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


array([[0., 0., 1., 0.]], dtype=float32)

In [133]:
model_json=model.to_json() # is a format popular in saving data like csv or xml style
json_file=open("Brain_tumor_model.json","w")
json_file.write(model_json)

model.save_weights("model.weights.h5") # to save in keras native  format
print("Saved model to disk")


Saved model to disk


In [135]:
from tensorflow.keras.models import model_from_json
json_file=open('Brain_tumor_model.json','r')
loaded_model_json=json_file.read()
json_file.close()

loaded_model=model_from_json(loaded_model_json) # load the saved model
loaded_model.load_weights("model.weights.h5")
print("Loaded model from disk")



Loaded model from disk


In [136]:
loaded_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
score=model.evaluate(validation_data_generator, steps=len(validation_data_generator))

# Print the evaluation score
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.9506 - loss: 0.2333
Test loss: 0.3249240517616272
Test accuracy: 0.9259259104728699


In [137]:
loaded_model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_24 (Conv2D)                   │ (None, 126, 126, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_24 (MaxPooling2D)      │ (None, 63, 63, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_36 (Dropout)                 │ (None, 63, 63, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_25 (Conv2D)                   │ (None, 61, 61, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_25 (MaxPooling2D)      │ (None, 30, 30, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_37 (Dropout)                 │ (None, 30, 30, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_12 (Flatten)                 │ (None, 28800)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 64)                  │       1,843,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_38 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 4)                   │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,848,612 (7.05 MB)

 Trainable params: 1,848,612 (7.05 MB)

 Non-trainable params: 0 (0.00 B)

In [138]:

# Prepare the image for prediction
import os
img_path1 = os.path.join('output', 'val', 'yes', 'Y2.jpg')  # Adjusted for relative path
img_yes= image.load_img(img_path1, target_size=(img_height, img_width))
img_arr = image.img_to_array(img_yes)
img_arr = np.expand_dims(img_arr, axis=0)

result=model.predict(img_arr)
result


# Decode the prediction to a meaningful class label

classes = {0: 'glioma_tumor', 1: 'meningioma_tumor', 2: 'no_tumor', 3: 'pituitary_tumor'}
predicted_class = np.argmax(result)
print(f"Predicted Class: {classes[predicted_class]} ({result})")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Predicted Class: pituitary_tumor ([[0. 0. 0. 1.]])
